In [1]:
import torch.nn.functional

from model import UNet
from lfw_dataset import LFWDataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
import cv2
import matplotlib.pyplot as plt
from utils import transform_generator, inv_transform
from copy import deepcopy
import pickle
from utils import eval
from train import train
import wandb

/home/georgerapeanu/miniconda3/envs/AI/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'NUM_EPOCHS': {
        'values': [10]
    },
     'BATCH_SIZE': {
         'values': [32, 48, 64, 80, 96]
      },
    'INPUT_SHAPE': {
        'values': [(64, 64)]
    },
    'NUM_LAYERS': {
        'values': [2]
    },
    'LR': {
        'values': [0.004, 0.008, 0.01]
      },
    'INTERMEDIARY_FILTERS': {
        'values': [8]
    }
}

# parameters_dict = {
#     'NUM_EPOCHS': {
#         'values': [10, 20]
#     },
#      'BATCH_SIZE': {
#         # integers between 32 and 256
#         # with evenly-distributed logarithms 
#         'distribution': 'q_log_uniform_values',
#         'q': 8,
#         'min': 32,
#         'max': 96,
#       },
#     'INPUT_SHAPE': {
#         'values': [(64, 64)]
#     },
#     'NUM_LAYERS': {
#         'values': [1, 2]
#     },
#     'LR': {
#         # a flat distribution between 0 and 0.1
#         'distribution': 'uniform',
#         'min': 0.004,
#         'max': 0.04
#       },
#     'INTERMEDIARY_FILTERS': {
#         'values': [8, 16, 32]
#     }
# }

sweep_config['parameters'] = parameters_dict

In [3]:
sweep_id = wandb.sweep(sweep_config, project="cvdl")

Create sweep with ID: cstsi9oz
Sweep URL: https://wandb.ai/georgerapeanu/cvdl/sweeps/cstsi9oz


In [4]:
def run_wandb():
    with wandb.init():
        train(None, wandb.config)

In [ ]:
wandb.agent(sweep_id, run_wandb, project="cvdl")RuntimeError: Can't redefine method: forward on class: __torch__.model.___torch_mangle_35.UNet (of Python compilation unit at: 0x5638a008b5a0)

In [2]:
wandb.init(
    # set the wandb project where this run will be logged
    project="cvdl",
    
    # track hyperparameters and run metadata
    config={
        'NUM_EPOCHS': 30,
        'BATCH_SIZE': 48,
        'INPUT_SHAPE': (128, 128),
        'NUM_LAYERS': 2,
        'LR': 0.004,
        'INTERMEDIARY_FILTERS': 32,
        'OPTIMIZER': 'Adam'
    }
)


wandb: Currently logged in as: georgerapeanu. Use `wandb login --relogin` to force relogin


In [3]:
wandb.config


{'NUM_EPOCHS': 10, 'BATCH_SIZE': 32, 'INPUT_SHAPE': [64, 64], 'NUM_LAYERS': 1, 'LR': 0.0713}

In [5]:
NUM_EPOCHS = 100
BATCH_SIZE= 128
INPUT_SHAPE = (64, 64)
NUM_LAYERS = 2
LR = 0.01

ARTIFACTS_PATH='./artifacts'
BASE_PATH="./lfw_dataset"

In [3]:
model = train(None, wandb.config)[0]

Epoch [1/30], Train Loss: 0.6673, Validation Loss: 0.5888
Epoch [2/30], Train Loss: 0.4759, Validation Loss: 0.5362
Epoch [3/30], Train Loss: 0.4283, Validation Loss: 0.7293
Epoch [4/30], Train Loss: 0.3908, Validation Loss: 0.4787
Epoch [5/30], Train Loss: 0.3735, Validation Loss: 0.6504
Epoch [6/30], Train Loss: 0.3548, Validation Loss: 0.6445
Epoch [7/30], Train Loss: 0.3263, Validation Loss: 0.4835
Epoch [8/30], Train Loss: 0.3072, Validation Loss: 0.3359
Epoch [9/30], Train Loss: 0.2871, Validation Loss: 0.4650
Epoch [10/30], Train Loss: 0.2850, Validation Loss: 0.3308
Epoch [11/30], Train Loss: 0.2622, Validation Loss: 0.3699
Epoch [12/30], Train Loss: 0.2444, Validation Loss: 0.3653
Epoch [13/30], Train Loss: 0.2359, Validation Loss: 0.3111
Epoch [14/30], Train Loss: 0.2320, Validation Loss: 0.2700
Epoch [15/30], Train Loss: 0.2234, Validation Loss: 0.2684
Epoch [16/30], Train Loss: 0.2131, Validation Loss: 0.2620
Epoch [17/30], Train Loss: 0.2080, Validation Loss: 0.2830
Epoch 

wandb: Waiting for artifact model_val_loss to be committed...

wandb: Committed artifact georgerapeanu/cvdl/model_val_loss:v425
wandb: Waiting for artifact model_mean_pixel_accuracy to be committed...

wandb: Committed artifact georgerapeanu/cvdl/model_mean_pixel_accuracy:v348
wandb: Waiting for artifact model_mean_intersection_over_union to be committed...

wandb: Committed artifact georgerapeanu/cvdl/model_mean_intersection_over_union:v352
wandb: Waiting for artifact model_fw_intersection_over_union to be committed...

wandb: Committed artifact georgerapeanu/cvdl/model_fw_intersection_over_union:v355


In [ ]:
model = train(model, {
        'NUM_EPOCHS': 60,
        'BATCH_SIZE': 48,
        'INPUT_SHAPE': (128, 128),
        'NUM_LAYERS': 2,
        'LR': 0.001,
        'INTERMEDIARY_FILTERS': 32,
        'OPTIMIZER': 'SGD'
    })[0]

Epoch [1/60], Train Loss: 0.1573, Validation Loss: 0.2443
Epoch [2/60], Train Loss: 0.1556, Validation Loss: 0.2409
Epoch [3/60], Train Loss: 0.1551, Validation Loss: 0.2387
Epoch [4/60], Train Loss: 0.1563, Validation Loss: 0.2356
Epoch [5/60], Train Loss: 0.1560, Validation Loss: 0.2312
Epoch [6/60], Train Loss: 0.1543, Validation Loss: 0.2335
Epoch [7/60], Train Loss: 0.1541, Validation Loss: 0.2306
Epoch [8/60], Train Loss: 0.1556, Validation Loss: 0.2269
Epoch [9/60], Train Loss: 0.1544, Validation Loss: 0.2360
Epoch [10/60], Train Loss: 0.1529, Validation Loss: 0.2368
Epoch [11/60], Train Loss: 0.1528, Validation Loss: 0.2341
Epoch [12/60], Train Loss: 0.1533, Validation Loss: 0.2274
Epoch [13/60], Train Loss: 0.1534, Validation Loss: 0.2306
Epoch [14/60], Train Loss: 0.1517, Validation Loss: 0.2268
Epoch [15/60], Train Loss: 0.1506, Validation Loss: 0.2322
Epoch [16/60], Train Loss: 0.1520, Validation Loss: 0.2345
Epoch [17/60], Train Loss: 0.1519, Validation Loss: 0.2286
Epoch 

In [9]:
wandb.finish()

In [7]:
ds = LFWDataset(BASE_PATH, transforms=transform_generator(INPUT_SHAPE), split_name='test', download=False)
model.eval()

X, y = ds[2]
model_y = model(X.view(-1, *X.shape))
model_y = torch.nn.functional.interpolate(model_y, size=tuple(y.shape))
model_y = model_y.view(-1, *y.shape).argmax(dim=0)

_, model_y = inv_transform(X, model_y)
X, y = inv_transform(X, y)

print(X.shape, y.shape, model_y.shape)
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

axes[0].imshow(X, cmap='gray')
axes[0].set_title('Input')

axes[1].imshow(cv2.cvtColor(model_y, cv2.COLOR_BGR2RGB))
axes[1].set_title('Output')

axes[2].imshow(cv2.cvtColor(y, cv2.COLOR_BGR2RGB))
axes[2].set_title('Ground truth')

# Remove ticks and labels for a cleaner display
for ax in axes:
    ax.axis('off')

# Show the plot
plt.show()

KeyboardInterrupt: 

In [ ]:
eval(model, LFWDataset(BASE_PATH, transforms=transform_generator(INPUT_SHAPE), download=False, split_name='test'))

In [6]:
ds = LFWDataset(BASE_PATH, transforms=transform_generator(INPUT_SHAPE), split_name='test', download=False)

In [ ]:
len(ds)

In [8]:
ds[2][0].shape

torch.Size([3, 64, 64])

In [2]:
model = torch.load("./artifacts/model.h5", map_location=torch.device('cpu'))

In [3]:
model.eval()
scripted_model = torch.jit.script(model)
scripted_model.save("./artifacts/scripted_model.pt")

In [4]:
with torch.no_grad():
    model_y = model(ds[2][0].view(-1, *ds[2][0].shape))
    model_y = torch.nn.functional.interpolate(model_y, size=tuple(ds[2][0].shape[1:]))
    model_y = model_y.view(-1, *ds[2][1].shape).argmax(dim=0)
    X, model_y = inv_transform(ds[2][0], model_y)
    X, y = inv_transform(ds[2][0], ds[2][1])
    fig, axes = plt.subplots(1, 3, figsize=(10, 5))
    
    axes[0].imshow(X)
    axes[0].set_title('Input')
    
    axes[1].imshow(cv2.cvtColor(model_y, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Output')
    
    axes[2].imshow(cv2.cvtColor(y, cv2.COLOR_BGR2RGB))
    axes[2].set_title('Ground truth')
    
    # Remove ticks and labels for a cleaner display
    for ax in axes:
        ax.axis('off')
    
    # Show the plot
    plt.show() 

NameError: name 'ds' is not defined

In [8]:
model = train(None, {
        'NUM_EPOCHS': 1,
        'BATCH_SIZE': 48,
        'INPUT_SHAPE': (128, 128),
        'NUM_LAYERS': 2,
        'LR': 0.001,
        'INTERMEDIARY_FILTERS': 32,
        'OPTIMIZER': 'SGD'
    })[0]

FileNotFoundError: [Errno 2] No such file or directory: './lfw_dataset/parts_train.txt'

In [2]:
 model = UNet(in_channels=3, num_layers=2, num_classes=3, intermediary_filters=32)

In [6]:

total_params = sum(
	param.numel() for param in model.parameters()
)





In [7]:
total_params

478243